In [21]:
%matplotlib inline

# gensim modules
from gensim import utils
from smart_open import open
from gensim.models.doc2vec import LabeledSentence
from gensim.models import Doc2Vec

# numpy
import numpy as np


# classifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from sklearn.datasets import load_files
import pickle
from pyvi import ViTokenizer
from gensim.models import Word2Vec

In [22]:
import re
def text_preprocessing(document):
   
    replace_list = {
        'òa': 'oà', 'óa': 'oá', 'ỏa': 'oả', 'õa': 'oã', 'ọa': 'oạ', 'òe': 'oè', 'óe': 'oé','ỏe': 'oẻ',
        'õe': 'oẽ', 'ọe': 'oẹ', 'ùy': 'uỳ', 'úy': 'uý', 'ủy': 'uỷ', 'ũy': 'uỹ','ụy': 'uỵ', 'uả': 'ủa',
        'ả': 'ả', 'ố': 'ố', 'u´': 'ố','ỗ': 'ỗ', 'ồ': 'ồ', 'ổ': 'ổ', 'ấ': 'ấ', 'ẫ': 'ẫ', 'ẩ': 'ẩ',
        'ầ': 'ầ', 'ỏ': 'ỏ', 'ề': 'ề','ễ': 'ễ', 'ắ': 'ắ', 'ủ': 'ủ', 'ế': 'ế', 'ở': 'ở', 'ỉ': 'ỉ',
        'ẻ': 'ẻ', 'àk': u' à ','aˋ': 'à', 'iˋ': 'ì', 'ă´': 'ắ','ử': 'ử', 'e˜': 'ẽ', 'y˜': 'ỹ', 'a´': 'á',
        ' okey ': ' ok ', 'ôkê': ' ok ', 'oki': ' ok ', ' oke ':  ' ok ',' okay':' ok ','okê':' ok ',
        ' tks ': u' cám ơn ', 'thks': u' cám ơn ', 'thanks': u' cám ơn ', 'ths': u' cám ơn ', 'thank': u' cám ơn ',
        ' kg ': u' không ','not': u' không ', u' kg ': u' không ', ' k ': u' không ',' kh ':u' không ',' kô ':u' không ',' hok ':u' không ',' kp ': u' không phải ',u' kô ': u' không ', ' ko ': u' không ', u' ko ': u' không ', u' k ': u' không ', 'khong': u' không ', u' hok ': u' không ',' hong ':u' không',
        'vs': u' với ', 'wa': ' quá ', 'wá': u' quá', 'j': u' gì ', '“': ' ',' đx ': u' được ', 'dk': u' được ', 'dc': u' được ', 'đk': u' được ',
        'đc': u' được ',' thick ': u' thích ', 'store': u' cửa hàng ',
        'shop': u' cửa hàng ', 'sp': u' sản phẩm ', 'gud': u' tốt ','god': u' tốt ','wel done':' tốt ', 'good': u' tốt ', 'gút': u' tốt ',
        ' sấu ': u' xấu ','gut': u' tốt ', u' tot ': u' tốt ', u' nice ': u' tốt ', 'perfect': 'rất tốt', 'bt': u' bình thường ',
        'time': u' thời gian ', 'qá': u' quá ', u' ship ': u' giao hàng ', u' m ': u' mình ', u' mik ': u' mình ',
        'date': u' hạn sử dụng ', 'hsd': u' hạn sử dụng ','quickly': u' nhanh ', 'quick': u' nhanh ','fast': u' nhanh ','delivery': u' giao hàng ',u' síp ': u' giao hàng ',
        'beautiful': u' đẹp tuyệt vời ', u' tl ': u' trả lời ', u' r ': u' rồi ', u' shopE ': u' cửa hàng ',u' order ': u' đặt hàng ',
        'chất lg': u' chất lượng ',u' sd ': u' sử dụng ',u' dt ': u' điện thoại ',u' nt ': u' nhắn tin ',u' tl ': u' trả lời ',u' sài ': u' xài ',u'bjo':u' bao giờ ',
        'thik': u' thích ',u' sop ': u' cửa hàng ', ' fb ': ' facebook ', ' face ': ' facebook ', ' very ': u' rất ',u'quả ng ':u' quảng  ',
        ' dep ': u' đẹp ',u' xau ': u' xấu ','delicious': u' ngon ', u'hàg': u' hàng ', u'qủa': u' quả ','iu': u' yêu ','pv':u' phục vụ','nv':u' nhân viên','nc':u' nước',
        u' hó ':u' khó',u' hắc ':u' khắc ','_': ' ', u' ng ':u' người '}

    for k, v in replace_list.items():
        document = document.replace(k, v)
    # Loại bỏ Hashtag
    document = re.sub(r'#([^\s]+)', '', document)
    # Loại bỏ các URL
    document = re.sub(r'http([^\s]+)', '', document)
    # Loại bỏ number có trong comment
    document = re.sub(r'\d+', '', document)
    #Remove duplicated characters: vd: đẹppppppp
    document = re.sub(r'([A-Z])\1+', lambda m: m.group(1).upper(), document, flags=re.IGNORECASE)
    # Loại bỏ các dấu câu, các ký tự đặc biệt (Emoji, icon cũng bị remove)
    document = re.sub(r'[^\w\s]','', document)
    # Chuyển chữ in hoa thành chữ thường
    document = document.lower()
    # Loại bỏ các ký tự dư thừa đơn lẻ (Nhiều ký tự đơn lẻ liền nhau kg xử lý được)
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    # Loại bỏ các Emoji, icon
    document = re.sub(r'[^\w\s,]', '', document)
    # Loại bỏ nhiều khoảng trắng dư thừa
     #document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Tokenization
    document = ViTokenizer.tokenize(document)
    return document

In [23]:
# Đặt đường dẫn để mở tất cả các file .txt trong thư mục coment
from sklearn.model_selection import train_test_split
from pyvi import ViTokenizer
import glob

#==========================================================
# Tạo ra file NEG từ tất cả các file trong thư mục neg
neg_paths = glob.glob("./data2/neg/*.txt")
neg_comments = []

for path in neg_paths :
  with open(path,encoding="utf-8") as file:
      text= file.read()
      text = text_preprocessing(text)
      neg_comments.append(text)


# Chia dữ liệu thành 70:30 và lưu thành 2 file neg_train và ner_test
neg_train, neg_test = train_test_split(neg_comments, test_size = 0.3, random_state = 42)

with open("./doc2vec_data2/neg_train.txt", 'w', encoding="utf-8") as output:
    for row in neg_train:
        output.write(str(row) + '\n')
        
with open("./doc2vec_data2/neg_test.txt", 'w', encoding="utf-8") as output:
    for row in neg_test:
        output.write(str(row) + '\n')

In [24]:
#==========================================================

# Tạo ra file POS từ tất cả các file trong thư mục pos
pos_paths = glob.glob("./data2/pos/*.txt")
pos_comments = []

for path in pos_paths :
  with open(path,encoding="utf-8") as file:
      text= file.read()
      text = text_preprocessing(text)
      pos_comments.append(text)
        
# Chia dữ liệu thành 70:30 và lưu thành 2 file pos_train và pos_test
pos_train, pos_test = train_test_split(pos_comments, test_size = 0.3, random_state = 42)

with open("./doc2vec_data2/pos_train.txt", 'w', encoding="utf-8") as output:
    for row in pos_train:
        output.write(str(row) + '\n')
        
with open("./doc2vec_data2/pos_test.txt", 'w', encoding="utf-8") as output:
    for row in pos_test:
        output.write(str(row) + '\n')


# Xây dựng mô hình Doc2Vec

In [25]:
class LabeledLineSentence(object):
    def __init__(self, sources):
        self.sources = sources
        
        flipped = {}
        
        # make sure that keys are unique
        for key, value in sources.items():
            if value not in flipped:
                flipped[value] = [key]
            else:
                raise Exception('Non-unique prefix encountered')
    
    def __iter__(self):
        for source, prefix in self.sources.items():
            with open(source, encoding="utf-8") as fin: 
                for item_no, line in enumerate(fin):
                    yield LabeledSentence(utils.to_unicode(line).split(), [prefix + '_%s' % item_no])
    
    def to_array(self):
        self.sentences = []
        for source, prefix in self.sources.items():
            with open(source, encoding="utf-8") as fin:
                for item_no, line in enumerate(fin):
                    self.sentences.append(LabeledSentence(utils.to_unicode(line).split(), [prefix + '_%s' % item_no]))
        return self.sentences
    
    def sentences_perm(self):
        shuffled = list(self.sentences)
        random.shuffle(shuffled)
        return shuffled

In [26]:
# Ok bây giờ chúng ta feed data files vào `LabeledLineSentence`,
# `LabeledLineSentence` input 1 dict với key là tên file, value là `prefix` của các sentences trong văn bản.  
sources = {
    'doc2vec_data2/neg_test.txt':'TEST_NEG',
    'doc2vec_data2/pos_test.txt':'TEST_POS', 
    'doc2vec_data2/neg_train.txt':'TRAIN_NEG', 
    'doc2vec_data2/pos_train.txt':'TRAIN_POS'
}

sentences = LabeledLineSentence(sources)

In [27]:
# Building the Vocabulary Table

model = Doc2Vec(min_count=1, window=10, size=300, sample=1e-4, negative=5, workers=7)
model.build_vocab(sentences.to_array())

C:\Users\ADMIN\anaconda3\lib\site-packages\gensim\models\doc2vec.py:574: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
C:\Users\ADMIN\anaconda3\lib\site-packages\ipykernel_launcher.py:25: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).


In [28]:
#Inspecting the Model

#model.most_similar('phục_vụ')

# Tạo training data và được word embdding bằng Doc2Vec

In [29]:
# Tính tổng số training_data vì ở trên mình đã chia tỷ lệ 70:30
len_X_train = len(pos_train) + len(neg_train)

# Tạo training data và được word embdding bằng Doc2Vec
X_train = np.zeros((len_X_train, 300))
y_train = np.zeros(len_X_train)

# Ở đây lấy giá trị neg_train hay pos_train cũng được, vì số lượng pos và neg bằng nhau
for i in range(len(neg_train)):
    prefix_train_pos = 'TRAIN_POS_' + str(i)
    prefix_train_neg = 'TRAIN_NEG_' + str(i)
    X_train[i] = model.docvecs[prefix_train_pos]
    X_train[len(neg_train) + i] = model.docvecs[prefix_train_neg]
    y_train[i] = 1
    y_train[len(neg_train) + i] = 0

# Tạo testing data và được word embdding bằng Doc2Vec

In [30]:
# Tính tổng số testing_data vì ở trên mình đã chia tỷ lệ 70:30
len_X_test = len(pos_test) + len(neg_test)

# Tạo testing data và được word embdding bằng Doc2Vec
X_test = np.zeros((len_X_test, 300))
y_test = np.zeros(len_X_test)

# Ở đây lấy giá trị neg_test hay pos_test cũng được, vì số lượng pos và neg bằng nhau
for i in range(len(neg_test)):
    prefix_test_pos = 'TEST_POS_' + str(i)
    prefix_test_neg = 'TEST_NEG_' + str(i)
    X_test[i] = model.docvecs[prefix_test_pos]
    X_test[len(neg_test) + i] = model.docvecs[prefix_test_neg]
    y_test[i] = 1
    y_test[len(neg_test) + i] = 0

In [31]:
#X_test

In [32]:
# Xây dựng mô hình huấn luyện bằng NaiveBayes

import time
from sklearn.naive_bayes import BernoulliNB

# Thêm biến này vào để kiểm tra thời gian huấn luyện
start_NB = time.time()


classifier_NB = BernoulliNB(binarize=None)
classifier_NB.fit(X_train, y_train)

end_NB = time.time()
time_NB = end_NB-start_NB

#Show ra thời gian huấn luyện của mô hình
#print("Thời gian huấn luyện: %s second" % str(end-start))
print("Thời gian huấn luyện: ", str(time_NB), " giây")

Thời gian huấn luyện:  0.009993791580200195  giây


In [33]:
# Xây dựng mô hình huấn luyện bằng Random Forest
import time
from sklearn.ensemble import RandomForestClassifier

# Thêm biến này vào để kiểm tra thời gian huấn luyện
start_RF = time.time()

classifier_RF = RandomForestClassifier(n_estimators=1000, random_state=0)
classifier_RF.fit(X_train, y_train) 

end_RF = time.time()
time_RF = end_RF-start_RF

#Show ra thời gian huấn luyện của mô hình
#print("Thời gian huấn luyện: %s second" % str(end-start))
print("Thời gian huấn luyện: ", str(time_RF), " giây")

Thời gian huấn luyện:  192.6567759513855  giây


In [34]:
# Xây dựng mô hình huấn luyện bằng SVM
import time
from sklearn.svm import SVC

# Thêm biến này vào để kiểm tra thời gian huấn luyện
start_SVM = time.time()

classifier_SVM = SVC(kernel = 'linear', random_state = 0)
classifier_SVM.fit(X_train, y_train)


end_SVM = time.time()
time_SVM = end_SVM-start_SVM

#Show ra thời gian huấn luyện của mô hình
#print("Thời gian huấn luyện: %s second" % str(end-start))
print("Thời gian huấn luyện: ", str(time_SVM), " giây")

Thời gian huấn luyện:  28.953555822372437  giây


In [35]:
# Xây dựng mô hình huấn luyện bằng Logistic Regression

from sklearn.linear_model import LogisticRegression
start = time.time()

classifier_LR = LogisticRegression(fit_intercept=True, n_jobs=4)
classifier_LR.fit(X_train, y_train)

end = time.time()

print("Thời gian huấn luyện Logistic Regression: ", str(end-start), " giây")

Thời gian huấn luyện Logistic Regression:  1.6249902248382568  giây


In [36]:
# Xây dựng mô hình huấn luyện bằng KNN

from sklearn.neighbors import KNeighborsClassifier

start = time.time()

classifier_KNN = KNeighborsClassifier(n_neighbors=5)
classifier_KNN.fit(X_train, y_train)

end = time.time()

print("Thời gian huấn luyện KNN: ", str(end-start), " giây")

Thời gian huấn luyện KNN:  2.6323866844177246  giây


In [37]:
# Predicting Sentiment

y_pred_NB = classifier_NB.predict(X_test)

y_pred_RF = classifier_RF.predict(X_test)

y_pred_SVM = classifier_SVM.predict(X_test)

y_Pred_LR = classifier_LR.predict(X_test)

y_Pred_KNN = classifier_KNN.predict(X_test)

In [38]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print('Accuracy LogisticRegression: ', accuracy_score(y_test, y_Pred_LR))
print('Accuracy of Naive Bayes:',accuracy_score(y_test, y_pred_NB))
print('Accuracy KNN: ', accuracy_score(y_test, y_Pred_KNN))
print('Accuracy of Random Forest:',accuracy_score(y_test, y_pred_RF))
print('Accuracy of SVM:',accuracy_score(y_test, y_pred_SVM))

Accuracy LogisticRegression:  0.4866666666666667
Accuracy of Naive Bayes: 0.49233333333333335
Accuracy KNN:  0.5186666666666667
Accuracy of Random Forest: 0.489
Accuracy of SVM: 0.48733333333333334
